# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,atafu village,-8.5421,-172.5159,28.53,79,11,4.50,TK,1746951356
1,1,isafjordur,66.0755,-23.1240,2.02,60,0,1.54,IS,1746951357
2,2,mandal,25.4500,74.5667,37.51,24,4,3.70,IN,1746951358
3,3,cabo san lucas,22.8909,-109.9124,22.96,51,44,5.07,MX,1746951359
4,4,minas de marcona,-15.2119,-75.1103,13.19,88,18,3.34,PE,1746951360


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM", # You can try other tiles like "CartoLight", "EsriImagery"
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8, # Adjust scale as needed
    color = "City",
    alpha = 0.6, # Adjust alpha for better visibility if points overlap
    hover_cols = ["City", "Humidity"]
)

# Display the map
map_plot_1



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
#A max temperature lower than 27 degrees but higher than 21
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]

#Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

#Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop results with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
139,139,maun,-19.9833,23.4167,24.42,22,0,2.68,BW,1746951764
155,155,luwingu,-10.2621,29.9271,22.09,64,0,3.24,ZM,1746951782
167,167,imperatriz,-5.5264,-47.4917,23.03,100,0,0.00,BR,1746951794
184,184,carlsbad,33.1581,-117.3506,21.71,86,0,1.54,US,1746951812
210,210,gunjur,13.2019,-16.7339,24.04,60,0,2.57,GM,1746951842
225,225,namibe,-15.1961,12.1522,25.46,58,0,1.89,AO,1746951858
261,261,betafo,-19.8333,46.8500,23.32,60,0,2.24,MG,1746951896
396,396,beira,-19.8436,34.8389,25.01,71,0,2.56,MZ,1746952044
424,424,boa vista,2.8197,-60.6733,25.99,83,0,1.03,BR,1746952076
447,447,tsiroanomandidy,-18.7667,46.0333,25.72,60,0,3.37,MG,1746952102


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_df = pd.DataFrame({
"City": city,
"Country": country,
"Lat": lat,
"Lng": lng,
"Humidity": humidity
})

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
139,maun,BW,-19.9833,23.4167,22,
155,luwingu,ZM,-10.2621,29.9271,64,
167,imperatriz,BR,-5.5264,-47.4917,100,
184,carlsbad,US,33.1581,-117.3506,86,
210,gunjur,GM,13.2019,-16.7339,60,
225,namibe,AO,-15.1961,12.1522,58,
261,betafo,MG,-19.8333,46.8500,60,
396,beira,MZ,-19.8436,34.8389,71,
424,boa vista,BR,2.8197,-60.6733,83,
447,tsiroanomandidy,MG,-18.7667,46.0333,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]


    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()


    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
maun - nearest hotel: Center Lodge Conference Center
luwingu - nearest hotel: No hotel found
imperatriz - nearest hotel: Hotel Posseidon
carlsbad - nearest hotel: Carlsbad Inn Beach Resort
gunjur - nearest hotel: Footsteps Eco Lodge
namibe - nearest hotel: Nelsal Pensao
betafo - nearest hotel: AkoorYaby Betafo
beira - nearest hotel: Hotel Savoy
boa vista - nearest hotel: Aipana Plaza Hotel
tsiroanomandidy - nearest hotel: Mega Food Hotel
palapye - nearest hotel: Palapaye Hotel
huaihua - nearest hotel: 怀仁养生酒店
shahr-e kord - nearest hotel: هتل دامون
urumqi - nearest hotel: 百时快捷酒店红山店
atlantic beach - nearest hotel: One Ocean Resort & Spa
kaoma - nearest hotel: Pumulo Pardway Lodge


,City,Country,Lat,Lng,Humidity,Hotel Name
139,maun,BW,-19.9833,23.4167,22,Center Lodge Conference Center
155,luwingu,ZM,-10.2621,29.9271,64,No hotel found
167,imperatriz,BR,-5.5264,-47.4917,100,Hotel Posseidon
184,carlsbad,US,33.1581,-117.3506,86,Carlsbad Inn Beach Resort
210,gunjur,GM,13.2019,-16.7339,60,Footsteps Eco Lodge
225,namibe,AO,-15.1961,12.1522,58,Nelsal Pensao
261,betafo,MG,-19.8333,46.8500,60,AkoorYaby Betafo
396,beira,MZ,-19.8436,34.8389,71,Hotel Savoy
424,boa vista,BR,2.8197,-60.6733,83,Aipana Plaza Hotel
447,tsiroanomandidy,MG,-18.7667,46.0333,60,Mega Food Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1, # Adjust scale if needed
    color = "City",
    alpha = 0.6,
    hover_cols = ["Hotel Name", "Country", "City", "Humidity"] # Added City and Humidity for context
)


# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)